In [1]:
import pandas as pd
import numpy as np
import requests

In [2]:
website_url=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [3]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'lxml')
#print(soup.prettify())

In [4]:
My_table = soup.find('table',{'class':'wikitable sortable'})
links=My_table.findAll('tr')
#links

In [5]:
df = pd.DataFrame()
headers=My_table.findAll('th')
headings = [th.text.strip() for th in headers]    
headings

df = pd.DataFrame(columns=[headings[0], headings[1], headings[2]])

In [6]:
for link in links:
    tds = link.find_all('td')
    if len(tds) != 0:
        records = [td.text.strip() for td in tds]
        if records[2] =='Not assigned':
            records[2]=records[1]
        if records[1] != 'Not assigned':
            df=df.append({'Postcode': records[0],'Borough': records[1],'Neighbourhood': records[2]}, ignore_index=True)


In [7]:
df_new=df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(lambda x: "%s" % ', '.join(x))
length=len(df_new)


df_final = pd.DataFrame()
df_final = pd.DataFrame(columns=['Postcode', 'Borough', 'Neighbourhood'])
i=0
for i in np.arange(0, length):
    df_final = df_final.append({'Postcode': str(df_new.index[i][0]),'Borough': str(df_new.index[i][1]),'Neighbourhood': str(df_new[i])}, ignore_index=True)



In [8]:
!pip install geocoder
import geocoder # import geocoder

In [9]:
geocord=pd.read_csv('http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv')
geocord.rename(columns={'Postal Code': 'Postcode'}, inplace=True)

In [10]:
df_final_latlong=df_final.merge(geocord, on='Postcode', how='left')
df_final_latlong

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [11]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

Solving environment: ...working... done

# All requested packages already installed.



In [12]:
 !conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

Solving environment: ...working... done

# All requested packages already installed.



In [16]:
CLIENT_ID = 'HUL4CRUZCLMKXUWUXDJDGHPOKZ1BMTR0QBBJEVBRLFQRAM0Y' # your Foursquare ID
CLIENT_SECRET = '2OQQLDFTQX3WQHT1ODJIZAWXSV5PTDBI2PC5SMS1YWEVYLTR' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

#print('Your credentails:')
#print('CLIENT_ID: ' + CLIENT_ID)
#print('CLIENT_SECRET:' + CLIENT_SECRET)

In [17]:
neighborhood_latitude = df_toronto.loc[0,'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_toronto.loc[0,'Longitude'] # neighborhood longitude value

neighborhood_name = df_toronto.loc[0,'Neighbourhood'] # neighborhood name

#neighborhood_latitude='43.836125'
#neighborhood_longitude='-79.205636'
#print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, neighborhood_latitude, neighborhood_longitude))

In [18]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=HUL4CRUZCLMKXUWUXDJDGHPOKZ1BMTR0QBBJEVBRLFQRAM0Y&client_secret=2OQQLDFTQX3WQHT1ODJIZAWXSV5PTDBI2PC5SMS1YWEVYLTR&v=20180605&ll=43.806686299999996,-79.19435340000001&radius=500&limit=100'

In [19]:
import requests
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d578855bf7dde002c4b320a'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 1,
  'suggestedBounds': {'ne': {'lat': 43.8111863045, 'lng': -79.18812958073042},
   'sw': {'lat': 43.80218629549999, 'lng': -79.2005772192696}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bb6b9446edc76b0d771311c',
       'name': "Wendy's",
       'location': {'crossStreet': 'Morningside & Sheppard',
        'lat': 43.80744841934756,
        'lng': -79.19905558052072,
        'labeledLatLngs': [{'label': 'display'

In [20]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [21]:
from pandas.io.json import json_normalize
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.id','venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()
#nearby_venues.iloc[0]['id']

,id,name,categories,lat,lng
0,4bb6b9446edc76b0d771311c,Wendy's,Fast Food Restaurant,43.807448,-79.199056


In [24]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

1 venues were returned by Foursquare.


In [25]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng,
            v['venue']['id'],
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude',
                  'VenueID',
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


In [26]:
toronto_venues = getNearbyVenues(names=df_toronto['Neighbourhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, 

In [27]:
print(toronto_venues.shape)
toronto_venues.head()

(2235, 8)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,VenueID,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,4bb6b9446edc76b0d771311c,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,4c23d3aaf7ced13a5ed7216d,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,536bdf8f498ede143cca0072,Scarborough Historical Society,43.788755,-79.162438,History Museum
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,4b6074e3f964a5200fe729e3,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,4c62f34bde1b2d7fec89e370,G & G Electronics,43.765309,-79.191537,Electronics Store


In [98]:
#toronto_venues['Venue Category'].unique()
#check=toronto_venues[toronto_venues['Venue Category'].str.contains("Coffee")]
#check
df_coffee=toronto_venues[toronto_venues['Venue Category'].str.contains("Coffee")]
df_coffee.reset_index(drop=True)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,VenueID,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Woburn,43.770992,-79.216917,4cc1d28c06c254815ac18547,Starbucks,43.770037,-79.221156,Coffee Shop
1,Woburn,43.770992,-79.216917,5164bf50e4b0ca7ab1d9ed8b,Tim Hortons,43.770827,-79.223078,Coffee Shop
2,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029,4c6aa500c946e21ec432ed8e,Tim Hortons,43.726895,-79.266157,Coffee Shop
3,L'Amoreaux West,43.799525,-79.318389,4bd0cd2477b29c74b5838b82,Tim Hortons,43.799102,-79.318715,Coffee Shop
4,L'Amoreaux West,43.799525,-79.318389,4e4bcc4162e1ce23e579d8dc,Coffee Time,43.797952,-79.318678,Coffee Shop
5,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,50bceb3ae4b01dc9b287cc99,Aroma Espresso Bar,43.777700,-79.344652,Coffee Shop
6,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,4af5d36ef964a52062fd21e3,Starbucks,43.777757,-79.344451,Coffee Shop
7,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,502fc355e4b0b49bb5791038,Tim Hortons,43.777964,-79.344715,Coffee Shop
8,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,5567bf49498e711ea9c1f3fd,Tim Hortons,43.774965,-79.346285,Coffee Shop
9,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,4c3f205c80bc20a1d1ecab58,Tim Hortons,43.775249,-79.347740,Coffee Shop


In [29]:
df_coffee_count=df_coffee.groupby('Neighborhood').count()
df_coffee_count.reset_index()
df_coffee_count=df_coffee_count.drop(columns=[ 'Neighborhood Latitude','Neighborhood Longitude','Venue','Venue Latitude','Venue Longitude'],axis=1)
df_coffee_count=df_coffee_count.reset_index()
df_coffee_count.rename(columns={'Venue Category': '#ofCoffeeShops'})


,Neighborhood,VenueID,#ofCoffeeShops
0,"Adelaide, King, Richmond",8,8
1,"Albion Gardens, Beaumond Heights, Humbergate, ...",1,1
2,"Alderwood, Long Branch",1,1
3,"Bathurst Manor, Downsview North, Wilson Heights",2,2
4,"Bedford Park, Lawrence Manor East",2,2
5,Berczy Park,5,5
6,"Bloordale Gardens, Eringate, Markland Wood, Ol...",1,1
7,"Brockton, Exhibition Place, Parkdale Village",2,2
8,"CN Tower, Bathurst Quay, Island airport, Harbo...",1,1
9,"Cabbagetown, St. James Town",4,4


In [43]:
venue_id = '4bb6b9446edc76b0d771311c' # ID of Harry's Italian Pizza Bar
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url


'https://api.foursquare.com/v2/venues/4bb6b9446edc76b0d771311c?client_id=HUL4CRUZCLMKXUWUXDJDGHPOKZ1BMTR0QBBJEVBRLFQRAM0Y&client_secret=2OQQLDFTQX3WQHT1ODJIZAWXSV5PTDBI2PC5SMS1YWEVYLTR&v=20180605'

In [44]:
result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

5.9


In [46]:
#df_coffee

#def getratings(names, latitudes, longitudes, radius=500):
def getratings(vid):   
    rating_venues = pd.DataFrame(columns=['ID',  'VenueRating', 'NumberofRating'])
    for venueid in (vid):
        
        url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
        ###print(url)
            
     
        try:
            x=(results['response']['venue']['rating'])
        except:
            x=0
        try:
            y=(results['response']['venue']['ratingSignals'])
        except:
            y=0
    
        rating_venues = rating_venues.append({'ID':venue_id,  'VenueRating': x, 'NumberofRating': y}, ignore_index=True)
       
    return(rating_venues)

In [90]:
ratings=getratings(vid=df_coffee.loc[150:,'VenueID'])


In [91]:
ratings

,ID,VenueRating,NumberofRating
0,563d2f2dcd10bcf27ae37c3b,0,0
1,59cd51c71b0ea516e9e7b3aa,8.8,22
2,4b6b4340f964a520c9fc2be3,8.3,101
3,592c5bfb31fd147e47255fb3,8,11
4,54132b3b498ee9ca9332e189,8.2,48
5,5a3a846af62e0960e9364d11,8,8
6,4bf438f0e5eba5936d671f90,7.5,43
7,4b981b46f964a520c22c35e3,8,39
8,4adc55c6f964a520bd2b21e3,7.9,337
9,4b198909f964a52006df23e3,8,65


In [92]:
ratings.to_csv(r'File Name4.csv')

In [125]:
path="C:/Users/GAURAV/Downloads/RatingsCombined.csv"
df_manualrating=pd.read_csv(path)

In [126]:
df_manualrating

,VenueID,VenueRating,NumberofRating
0,4cc1d28c06c254815ac18547,6.9,2
1,5164bf50e4b0ca7ab1d9ed8b,6.2,0
2,4c6aa500c946e21ec432ed8e,6.1,15
3,4bd0cd2477b29c74b5838b82,6.0,5
4,4e4bcc4162e1ce23e579d8dc,0.0,0
5,50bceb3ae4b01dc9b287cc99,6.9,20
6,4af5d36ef964a52062fd21e3,6.8,57
7,502fc355e4b0b49bb5791038,6.2,2
8,5567bf49498e711ea9c1f3fd,6.4,5
9,4c3f205c80bc20a1d1ecab58,6.3,1


In [128]:
df_manualrating.loc[(df_manualrating['VenueRating']!=0) & (df_manualrating['NumberofRating']==0), 'NumberofRating' ] = 1

df_manualrating=df_manualrating[(df_manualrating.VenueRating!=0) & (df_manualrating['NumberofRating']!=0)]

df_manualrating

,VenueID,VenueRating,NumberofRating
0,4cc1d28c06c254815ac18547,6.9,2
1,5164bf50e4b0ca7ab1d9ed8b,6.2,1
2,4c6aa500c946e21ec432ed8e,6.1,15
3,4bd0cd2477b29c74b5838b82,6.0,5
5,50bceb3ae4b01dc9b287cc99,6.9,20
6,4af5d36ef964a52062fd21e3,6.8,57
7,502fc355e4b0b49bb5791038,6.2,2
8,5567bf49498e711ea9c1f3fd,6.4,5
9,4c3f205c80bc20a1d1ecab58,6.3,1
10,4aedfeadf964a52005d121e3,7.8,126


In [131]:
df_manualrating.drop_duplicates(inplace=True)
df_manualrating

,VenueID,VenueRating,NumberofRating
0,4cc1d28c06c254815ac18547,6.9,2
1,5164bf50e4b0ca7ab1d9ed8b,6.2,1
2,4c6aa500c946e21ec432ed8e,6.1,15
3,4bd0cd2477b29c74b5838b82,6.0,5
5,50bceb3ae4b01dc9b287cc99,6.9,20
6,4af5d36ef964a52062fd21e3,6.8,57
7,502fc355e4b0b49bb5791038,6.2,2
8,5567bf49498e711ea9c1f3fd,6.4,5
9,4c3f205c80bc20a1d1ecab58,6.3,1
10,4aedfeadf964a52005d121e3,7.8,126


In [139]:
df_coffeeshops = pd.merge(df_coffee, df_manualrating, on=['VenueID'])
df_coffeeshops= pd.merge(df_coffeeshops,toronto_merged, on=['Neighborhood'])
df_coffeeshops=df_coffeeshops[['Postcode','Borough','Neighborhood','Neighborhood Latitude','Neighborhood Longitude','VenueID','Venue','Venue Latitude','Venue Longitude','Venue Category','VenueRating','NumberofRating','Cluster Labels']]
df_coffeeshops.drop_duplicates(inplace=True)
df_coffeeshops

,Postcode,Borough,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,VenueID,Venue,Venue Latitude,Venue Longitude,Venue Category,VenueRating,NumberofRating,Cluster Labels
0,M1G,Scarborough,Woburn,43.770992,-79.216917,4cc1d28c06c254815ac18547,Starbucks,43.770037,-79.221156,Coffee Shop,6.9,2,1
1,M1G,Scarborough,Woburn,43.770992,-79.216917,5164bf50e4b0ca7ab1d9ed8b,Tim Hortons,43.770827,-79.223078,Coffee Shop,6.2,1,1
2,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029,4c6aa500c946e21ec432ed8e,Tim Hortons,43.726895,-79.266157,Coffee Shop,6.1,15,1
3,M1W,Scarborough,L'Amoreaux West,43.799525,-79.318389,4bd0cd2477b29c74b5838b82,Tim Hortons,43.799102,-79.318715,Coffee Shop,6.0,5,1
4,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,50bceb3ae4b01dc9b287cc99,Aroma Espresso Bar,43.777700,-79.344652,Coffee Shop,6.9,20,1
5,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,4af5d36ef964a52062fd21e3,Starbucks,43.777757,-79.344451,Coffee Shop,6.8,57,1
6,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,502fc355e4b0b49bb5791038,Tim Hortons,43.777964,-79.344715,Coffee Shop,6.2,2,1
7,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,5567bf49498e711ea9c1f3fd,Tim Hortons,43.774965,-79.346285,Coffee Shop,6.4,5,1
8,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,4c3f205c80bc20a1d1ecab58,Tim Hortons,43.775249,-79.347740,Coffee Shop,6.3,1,1
9,M2N,North York,Willowdale South,43.770120,-79.408493,4aedfeadf964a52005d121e3,Starbucks,43.768192,-79.413021,Coffee Shop,7.8,126,1


In [224]:
df_coffeeshops.to_csv(r'CoffeeShop.csv')

In [183]:
df_coffeeshops['TotalShopRating']=df_coffeeshops['VenueRating']*df_coffeeshops['NumberofRating']

,Postcode,Borough,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,VenueID,Venue,Venue Latitude,Venue Longitude,Venue Category,VenueRating,NumberofRating,Cluster Labels,TotalShopRating
0,M1G,Scarborough,Woburn,43.770992,-79.216917,4cc1d28c06c254815ac18547,Starbucks,43.770037,-79.221156,Coffee Shop,6.9,2,1,13.8
1,M1G,Scarborough,Woburn,43.770992,-79.216917,5164bf50e4b0ca7ab1d9ed8b,Tim Hortons,43.770827,-79.223078,Coffee Shop,6.2,1,1,6.2
2,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029,4c6aa500c946e21ec432ed8e,Tim Hortons,43.726895,-79.266157,Coffee Shop,6.1,15,1,91.5
3,M1W,Scarborough,L'Amoreaux West,43.799525,-79.318389,4bd0cd2477b29c74b5838b82,Tim Hortons,43.799102,-79.318715,Coffee Shop,6.0,5,1,30.0
4,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,50bceb3ae4b01dc9b287cc99,Aroma Espresso Bar,43.777700,-79.344652,Coffee Shop,6.9,20,1,138.0
5,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,4af5d36ef964a52062fd21e3,Starbucks,43.777757,-79.344451,Coffee Shop,6.8,57,1,387.6
6,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,502fc355e4b0b49bb5791038,Tim Hortons,43.777964,-79.344715,Coffee Shop,6.2,2,1,12.4
7,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,5567bf49498e711ea9c1f3fd,Tim Hortons,43.774965,-79.346285,Coffee Shop,6.4,5,1,32.0
8,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,4c3f205c80bc20a1d1ecab58,Tim Hortons,43.775249,-79.347740,Coffee Shop,6.3,1,1,6.3
9,M2N,North York,Willowdale South,43.770120,-79.408493,4aedfeadf964a52005d121e3,Starbucks,43.768192,-79.413021,Coffee Shop,7.8,126,1,982.8


In [210]:
ShopCounts=ShopCounts.rename(columns={"Venue": "#ofoutletsofbrand"})
ShopCounts=ShopCounts.rename(columns={"index": "Venue"})
ShopCounts

,Venue,#ofoutletsofbrand
0,Starbucks,67
1,Tim Hortons,38
2,Second Cup,5
3,Dineen @CommerceCourt,5
4,Pilot Coffee Roasters,5
5,Balzac's Coffee,3
6,M Square Coffee Co,3
7,Aroma Espresso Bar,3
8,Everyday Gourmet (Teas & Coffees),2
9,Starbucks (MaRS),2
